In [ ]:
from datetime import date
import pandas as pd
import re
import requests

## Pull data

In [ ]:
# As of Feb 4, 2021, the endpoint returns workshop data from the last two years - ~1300 nodes
url = "https://www.lib.ncsu.edu/api/workshops/all"

In [ ]:
user_activity_url = "https://www.lib.ncsu.edu/api/user_activities/all"

In [ ]:
r = requests.get(url)
r

In [ ]:
data = r.json()

In [ ]:
data[0]

In [ ]:
df = pd.DataFrame(data)
df.head()

Strip white spaces from the `field_non_library_instructor` column. 

In [ ]:
df["field_non_library_instructor"] = df["field_non_library_instructor"].str.strip()
df.head()

Remove HTML tags from `body`.

In [ ]:
def strip_html_tags(text: str) -> str:
    tag = re.compile("<.*?>")
    return re.sub(tag, "", text)


In [ ]:
df["body"] = df["body"].apply(strip_html_tags)
df.head()

Write to csv, with date.

In [ ]:
df.to_csv(f"all-workshops-{date.today()}.csv")

## Explore text

TODO:
- Consistency across workshops
    - Length of description
    - Consistent keywords?
    - Does title reflect body paragraph? (check keywords)
        - Does body contain words from the title? Does title contain words from body?
    - Consistency of workshop descriptions across related titles
        - If we have multiple introductions to python, are the descriptions similar?
        - Similar across intro to R vs intro python? 
- Lexical variety?


In [ ]:
from nltk import word_tokenize
import numpy as np
import spacy

Read in the most current csv of workshop data. We'll hardcode it for now, but could always parse the filenames, and get the most recent. 

In [ ]:
df = pd.read_csv("current-workshops-2021-01-20.csv")

In [ ]:
def naive_count_words(text: str) -> int:
    return len([word for word in word_tokenize(text)
        if word not in [".", ",", "?", "'", "\"", ":", ";"]])

In [ ]:
naive_count_words(df["body"][0])

We've done a naive pass to check word count minus punctuation. Let's compare it to spaCy's parsing, removing punctuation and spaces/newlines.

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
def spacy_count_words(text: str) -> int:
    doc = nlp(text)
    no_punct = [t for t in doc if t.is_punct == False]
    no_space = [t for t in no_punct if t.is_space == False]
    return len(no_space) 

In [ ]:
spacy_count_words(df["body"][0])

Create a new column in the dataframe that contains number of words in the description(`body`).

In [ ]:
df["body_word_count"] = df["body"].apply(spacy_count_words)

In [ ]:
df.head()

In [ ]:
avg_word_count = np.mean(df["body_word_count"])
avg_word_count

In [ ]:
lowest_word_count = df["body_word_count"].min()
lowest_word_count

In [ ]:
highest_word_count = df["body_word_count"].max()
highest_word_count

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Histogram of word counts to see distribution
df["body_word_count"].hist()

Let's figure out the most common words across the current future workshops. 

In [ ]:
from collections import Counter
import textacy

In [ ]:
corpus = textacy.Corpus(nlp, df["body"].to_list())
corpus

In [ ]:
corpus.n_docs

In [ ]:
corpus.n_tokens

In [ ]:
counts = corpus.word_counts(by="lower_")

In [ ]:
sorted(counts.items(), key=lambda x: x[1], reverse=True)[:20]

In [ ]:
counts["python"]

In [ ]:
counts["git"]

How much of this is a result of some descriptions being substantially longer than others?

TODO: check for how many workshops these commonly used terms appear in. 

In [ ]:
def check_inclusion_nums(token, corpus):
    count = 0
    for doc in corpus:
        if token in [token.text for token in doc]:
            count += 1
    return count

In [ ]:
check_inclusion_nums("IRB", corpus)

TODO: Write a function that takes a term and a corpus, and returns all the docs that have that term

Check to see if textacy already has this...

## Generate keywords for each description

In [ ]:
from textacy import extract
from itertools import chain

In [ ]:
textacy.extract.keyterms.sgrank(corpus[20])

In [ ]:
def get_keywords(text, spacy_model):
    doc = spacy_model(text)
    key_tuples = textacy.extract.keyterms.sgrank(doc)
    return [k for k, _ in key_tuples]

In [ ]:
df["keywords"] = df["body"].apply(get_keywords, spacy_model=nlp)

In [ ]:
df.head()

In [ ]:
df.to_csv("current-workshops-anno.csv")

In [ ]:
for i in range(len(df["title"])):
    print(df["title"][i], "--", df["keywords"][i])

## Titles and descriptions

Let's find out whether the descriptions use the same words as the title. 

In [ ]:
def check_title_body(title, body):
    title_words = title.split()
    words_in_body = []
    for word in title_words:
        if word in body:
            words_in_body.append(word)
    return words_in_body

In [ ]:
check_title_body(df["title"][0], df["body"][0])

In [ ]:
title_words_in_body = [check_title_body(df["title"][i], df["body"][i]) for i in range(len(df["title"]))]
df["title_words_in_body"] = title_words_in_body

In [ ]:
df.head()

In [ ]:
df.to_csv("current-workshops-anno.csv")

## Text summarization with transformers

Many of the descriptions are already short enough that they don't need summarization, but what can we see?

https://huggingface.co/transformers/main_classes/pipelines.html
https://huggingface.co/transformers/main_classes/pipelines.html#transformers.SummarizationPipeline

In [ ]:
from transformers import pipeline
from transformers import AutoModelWithLMHead, AutoTokenizer

In [ ]:
summarizer = pipeline("summarization")

In [ ]:
summarizer(df["body"][63])

In [ ]:
df["body"][63]

In [ ]:
summarizert5 = pipeline("summarization", model="t5-base", tokenizer="t5-base")

In [ ]:
summarizert5(df["body"][63], max_length=40)

In [ ]:
model = AutoModelWithLMHead.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

In [ ]:
inputs = tokenizer.encode("summarize: " + df["body"][63], return_tensors="pt", max_length=512)
outputs = model.generate(inputs, max_length=120, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

In [ ]:
tokenizer.decode(outputs[0])

In [ ]:
for desc in df["body"]:
    if len(desc.split()) > 200:
        print(summarizert5(desc))

## Question answering

Can we treat the whole of the descriptions (and titles?) as the context for a question answering task?

https://huggingface.co/transformers/task_summary.html#extractive-question-answering
https://huggingface.co/transformers/main_classes/pipelines.html#questionansweringpipeline

In [ ]:
all_descriptions = " ".join(df["body"])
len(all_descriptions.split())

In [ ]:
nlp = pipeline("question-answering")

In [ ]:
result = nlp(question="python?", context=all_descriptions)
print(f"Answer: {result['answer']}")

In [ ]:
result = nlp(question="Do you teach Python", context=all_descriptions)
print(f"Answer: {result['answer']}")